In [ ]:
!pip install aperturedb tqdm
from aperturedb import Utils
c = Utils.create_connector()

In [ ]:
u = Utils.Utils(c)
u.summary()

In [ ]:
# Now we retrieve the items we are working with:
# Retrieve the YOLO4 interface
!rm yolo4.py
!wget https://raw.githubusercontent.com/drewaogle/YOLOv4-OpenCV-CUDA-DNN/refs/heads/main/yolo4.py
# Retreive video
#!wget https://aperturedata-public.s3.us-west-2.amazonaws.com/aperturedb_applications/norman.mp4


In [ ]:

from importlib import reload
import yolo4
reload(yolo4)
from yolo4 import RemoteYOLOv4
class DetectorOptions:
    image='' # path for images
    stream='' # path for stream
    cfg="models/yolov4.cfg" # path to config
    weights="models/yolov4.weights" # path to weights
    namesfile="models/coco.names" # path for output to name mapping
    input_size=416
    use_gpu=False # use GPU or not
    outdir="output/norman"
    no_squash_detections=True
    def __init__(self, image='',stream=''):
        self.image = image
        self.stream=stream # 'webcam' to open webcam w/ OpenCV

# now we pull data
dopts = DetectorOptions( stream="norman.mp4")
yolo = RemoteYOLOv4.__new__(RemoteYOLOv4)
yolo.__init__(dopts)


In [ ]:
#Now let't check detections
import pandas as pd
df = pd.read_csv("output/norman/detections.csv")
print(df)

In [ ]:
# Fist we'll define the options we're going to use.
class ClipOptions:
    offset_frame=0 # starting offset in frames
    end_frame=-1 # ending offset in frames
    initconf=50 # minimun confidence to start ( 0-100 )
    initlen=5 # minimum detection duration in frames to start a clip
    dropconf=25 # confidence to end a frame (0 -100 )
    droplen=5 # number of detection missed frames to end a clip
    detections="output/noman/detections.csv" # path to output detections
    verbose=False # lots of info
    flush=False # remove old uuids
    nosave=False # dont add data to db
    label="" # label for video
    def __init__(self,video):
        self.video=video # video file to add

opts = ClipOptions( "norman.mp4" )
opts.label="Norman_Bike"
opts.initconf=45
opts.initlen=3
opts.dropconf=20
opts.droplen=3

In [ ]:
# define a function
def preprocess(df, args ):
   processed = df
   processed.columns = ["frame","label","confidence","left","top","width","height" ]
   processed.drop(processed[processed.frame < args.offset_frame].index, inplace=True)
   if args.end_frame > -1:
      processed.drop(processed[processed.frame > args.end_frame].index,inplace=True)
   return processed

C = preprocess( df, opts )
print(norman_detects)


In [ ]:
# process a frame by hand here.
from IPython.display import display as ds
import cv2
from PIL import Image

def display_image_and_bb( num, df ):
                    
    cv_image = cv2.imread( f"output/norman/video{num}.jpg")

    # Draw a rectangle around the faces
    counter = 0
    for id,coords in df[df["frame"] == num].iterrows():
        left   = coords["left"]
        top    = coords["top"]
        right  = coords["left"] + coords["width"]
        bottom = coords["top"] + coords["height"]
        cv2.rectangle(cv_image, (left, top), (right, bottom), (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(cv_image, coords["label"], (left, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        counter += 1

    cv_image_rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
    ds(Image.fromarray(cv_image_rgb))



In [ ]:
display_image_and_bb( 150, norman_detects)

In [ ]:
# process events which trigger on new frame.
def process_new_frame( verbose, cur_frame, last_frame, active, registered,finished):
           # drop any which werent active last frame
           new_active = {}
           new_registered = {}
           for old_label in active:
               last_active = active[old_label][0] + active[old_label][1]
               if cur_frame != last_active +1:
                   if verbose:
                       print(f'At frame {cur_frame}, Dropped {old_label} ( last active {last_active} )')
               else:   
                   if verbose:
                       print(f"At frame {cur_frame}, kept {old_label}")
                   new_active[old_label] = active[old_label]
           for old_label in registered:
               last_active = registered[old_label][0] + registered[old_label][1]
               total_active = registered[old_label][1]
               start_frame = registered[old_label][0]
               if cur_frame != last_active +1:
                   if verbose:
                       print(f'At frame {cur_frame}, Retired {old_label} frame duration: {total_active}, started {start_frame}')
                   finished[ "{}_{}".format(old_label,last_active) ] = [old_label] + registered[old_label]
               else:
                   new_registered[old_label] = registered[old_label]
           active = new_active
           registered = new_registered
           
           if verbose:
               print(f"Active dict: {active}")
    

In [ ]:
# process a row in the detections
# YOLOv4 can detect mutitple objects in a frame - this is a single detection in a given frame.
def process_row(verbose, initconf, initlen, dropconf, cur_frame, label, active, registered, confidence):
           # all old active and registered are dropped
       if label in active:
           label_start = active[label][0]
           label_len = active[label][1] + 1
           if confidence * 100 > initconf:
               max_confidence = max(active[label][2], confidence)
               if active[label][1] +1 >= initlen:
                   if verbose:
                       print(f"At frame {cur_frame}, moved {label} to registered")
                   registered[label] = [label_start,label_len,max_confidence]
               else:
                   if verbose:
                       print(f"At frame {cur_frame}, saw {label}")
                   active[label] = [label_start,label_len,max_confidence]
           else:   
               if verbose:
                   print(f"At frame {cur_frame}, saw {label}, but confidence was too low to re-register")
       elif label in registered:
           label_start = registered[label][0]
           label_len = registered[label][1]
           max_confidence = max(registered[label][2], confidence)
           # if above confidence for dropping, consider a new registered frame
           if confidence * 100 > dropconf:
                # allows frame to miss one and restart; duration calculated from start to current.
                registered[label] = [label_start,cur_frame-label_start,max_confidence] 
       else:    
           # if label not in active list
           if confidence * 100 > initconf:
               if verbose: 
                   print(f"At frame {cur_frame}, added {label}")
               active[label] = [cur_frame,0,confidence]
    

In [ ]:
# main loop over a frame.
def process(args,pf):
    active = {}
    registered = {}
    finished = {}
    last_frame =0
    cur_frame = 0
    for idx,row in pf.iterrows():
        cur_frame = row['frame']
        label = row['label']
        if cur_frame != last_frame:
           if args.verbose:
               print(f"Processing switch from {last_frame} to {cur_frame}")
           process_new_frame(args.verbose, cur_frame, last_frame, active, registered,finished)

        process_row(args.verbose, args.initconf, args.initlen, args.dropconf,cur_frame,label,active,registered,row['confidence'])

        last_frame = cur_frame
    return registered,finished
       

    

In [ ]:
norman_registered,norman_finished = process(opts,norman_detects)
print(norman_registered)
print(norman_finished)